In [3]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
%matplotlib inline
le = LabelEncoder()

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
test = pd.read_csv('SF_crime/test.csv', index_col='Id')
test = test.rename(columns={'X': 'Longitude', "Y": "Latitude"})
test.Dates = pd.to_datetime(test.Dates)
test_keep = test
crime_in_sf = pd.read_csv('SF_crime/train.csv')
crime_in_sf.Dates = pd.to_datetime(crime_in_sf.Dates)
crime_in_sf = crime_in_sf.rename(columns={'X': 'Longitude', "Y": "Latitude",})
crime_in_sf = crime_in_sf.drop(['Resolution', 'Descript'], axis=1)
crime_train, crime_test = train_test_split(crime_in_sf, test_size=.4)

In [5]:
for column in test.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(test[column])
        test[column] = le.transform(test[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])
        crime_train[column] = le.transform(crime_train[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])        
        crime_test[column] = le.transform(crime_test[column])

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
categories = crime_train.Category
crime_train = crime_train.drop('Category', axis=1)

categories2 = crime_test.Category
crime_test = crime_test.drop('Category', axis=1)

In [7]:
dtrain = xgb.DMatrix(crime_train.as_matrix(),
                     label=categories)
dtest = xgb.DMatrix(crime_test.as_matrix(),
                    label=categories2)

In [8]:
param = {'bst:max_depth':6, 'objective':'multi:softprob', 'num_class':39}
param['nthread'] = 4
param['eval_metric'] = ['merror', 'mlogloss']

In [9]:
evallist  = [(dtest,'eval'), (dtrain,'train')]

In [ ]:
num_round = 1000
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=5)

Will train until train error hasn't decreased in 5 rounds.
Multiple eval metrics have been passed: 'mlogloss' will be used for early stopping.

[0]	eval-merror:0.741979	eval-mlogloss:3.104986	train-merror:0.742508	train-mlogloss:3.102232


In [10]:
predictions = bst.predict(xgb.DMatrix(test.as_matrix()), output_margin=False)

In [11]:
predictions = pd.DataFrame(predictions)

In [12]:
le.fit(crime_in_sf.Category)
predictions.columns = le.inverse_transform(predictions.columns)

In [13]:
pred_test = predictions

In [14]:
predictions['Id'] = predictions.index

In [15]:
def order(frame,var):
    varlist =[w for w in frame.columns if w not in var]
    frame = frame[var+varlist]
    return frame
predictions = order(predictions,['Id'])

In [16]:
predictions.to_csv('predictions_XGB1000.csv', index=False)

In [17]:
def dummy_to_column(df):
    columns = df.columns.values
    df_t = df.T
    characters = []
    for column in range(len(columns)):
        for row in range(len(df[columns[column]])):
            if df[columns[column]][row] == '1':
                df[columns[column]][row] = columns[column]
    columns = df_t.columns.values
    characters = []
    for column in range(len(columns)):
        for row in range(len(df_t[columns[column]])):
            if df_t[columns[column]][row] in df_t.index:
                characters.append(df_t[columns[column]][row])
    return characters

def prob_to_column(df):
    df_t = df.T
    columns = df_t.columns.values
    probability = []
    for column in range(len(columns)):
        current_prob = 0
        for row in range(len(df_t[columns[column]])):
            if df_t[columns[column]][row] > current_prob:
                current_prob = df_t[columns[column]][row]
        probability.append(current_prob)
    n = 0
    df_t = df_t.applymap(str)
    for column in range(len(columns)):
        for row in range(len(df_t[column])):
            if df_t[column][row] == str(probability[n]):
                df_t[column][row] = '1'
        n+=1
    return df_t.T

In [21]:
crime_pred = pd.read_csv('predictions_XGB2.csv', index_col='Id')

In [ ]:
crime_pred = dummy_to_column(prob_to_column(crime_pred))

In [ ]:
crime_pred = pd.DataFrame(crime_pred)

In [ ]:
crime_pred.to_csv('crime_pred.csv')

In [ ]:
pred_results = pd.DataFrame(dummy_to_column(prob_to_column(pred_test)))

In [ ]:
test = test.join(pred_results)